# Concurrency and Parallelism

**Concurrency** is when a computer does many different things *seemingly* at the same time.

**Parallelism** is *actually* doing many different things at the same time.

The key difference between parallelism and concurrency is *speedup*.

Python makes it easy to write concurrent programs. But it can be very difficult to make concurrent Python code truly run in parallel.

- [tem 36: Use subprocess to Manage Child Processes](#Item-36:-Use-subprocess-to-Manage-Child-Processes)

## Item 36: Use *subprocess* to Manage Child Processes

With the Python of today, the best and simplest choice for managing child processes is to use the *subprocess* built-in module.

In [ ]:
import subprocess

proc = subprocess.Popen(
        ['echo', 'Hello from the child'],
        stdout=subprocess.PIPE)
out, err = proc.communicate()
print(out.decode('utf-8'))

print('starting')
proc = subprocess.Popen(['sleep', '1'])
print('started')
proc.communicate()
print('communicated')
while proc.poll() is None:
    pass
    
print('Exit status', proc.poll())

In [ ]:
from time import time

def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

start = time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)
    
for proc in procs:
    proc.communicate()
end = time()
print('Finished in %.3f seconds' % (end - start))

In [ ]:
import os

def run_openssl(data):
    env = os.environ.copy()
    env['password'] = b'\xe24U\n\xd0Ql3S\x11'
    proc = subprocess.Popen(
        ['openssl', 'enc', '-des3', '-pass', 'env:password'],
        env=env,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush()
    return proc

procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    procs.append(proc)
    
for proc in procs:
    out, err = proc.communicate()
    print(out[-10:])
    
def run_md5(input_stdin):
    proc = subprocess.Popen(
        ['md5'],
        stdin=input_stdin,
        stdout=subprocess.PIPE)
    return proc

input_procs = []
hash_procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    input_procs.append(proc)
    hash_proc = run_md5(proc.stdout)
    hash_procs.append(hash_proc)
    
for proc in input_procs:
    proc.communicate()
for proc in hash_procs:
    out, err = proc.communicate()
    print(out.strip())

In [ ]:
def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

proc = run_sleep(10)
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()
    
print('Exit status', proc.poll())

### Things to Remember

- Use the *subprocess* module to run child processes and manage their input and output streams.
- Child processes run in parallel with the Python interpreter, enabling you to maximize your CPU usage.
- Use the *timeout* parameter with *communicate* to avoid deadlocks and hanging child processes.